# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 28 2022 3:22PM,240668,10,MSP211712,"Methapharm, Inc.",Released
1,Jun 28 2022 3:22PM,240668,10,MSP211713,"Methapharm, Inc.",Released
2,Jun 28 2022 3:22PM,240668,10,MSP211722,"Methapharm, Inc.",Released
3,Jun 28 2022 3:22PM,240667,10,MSP211721,"Methapharm, Inc.",Released
4,Jun 28 2022 3:22PM,240667,10,MSP211711,"Methapharm, Inc.",Released
5,Jun 28 2022 3:22PM,240667,10,MSP211714,"Methapharm, Inc.",Released
6,Jun 28 2022 3:22PM,240667,10,MSP211715,"Methapharm, Inc.",Released
7,Jun 28 2022 3:22PM,240667,10,MSP211717,"Methapharm, Inc.",Released
8,Jun 28 2022 3:22PM,240667,10,MSP211718,"Methapharm, Inc.",Released
9,Jun 28 2022 3:22PM,240667,10,MSP211719,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,240663,Released,1
43,240664,Released,1
44,240665,Released,1
45,240667,Released,8
46,240668,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240663,NaN,NaN,1.0
240664,NaN,NaN,1.0
240665,NaN,NaN,1.0
240667,NaN,NaN,8.0
240668,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240574,0.0,0.0,1.0
240577,0.0,0.0,1.0
240579,0.0,0.0,1.0
240589,0.0,11.0,15.0
240591,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240574,0,0,1
240577,0,0,1
240579,0,0,1
240589,0,11,15
240591,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240574,0,0,1
1,240577,0,0,1
2,240579,0,0,1
3,240589,0,11,15
4,240591,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240574,,,1
1,240577,,,1
2,240579,,,1
3,240589,,11,15
4,240591,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc."
3,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc."
11,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc."
12,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc."
13,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc."
14,Jun 28 2022 2:58PM,240662,18,"ClearSpec, LLC"
15,Jun 28 2022 2:53PM,240661,20,Alumier Labs Inc.
27,Jun 28 2022 2:33PM,240660,10,ISDIN Corporation
40,Jun 28 2022 2:28PM,240658,18,"ClearSpec, LLC"
41,Jun 28 2022 2:25PM,240657,12,Doctors Scientific Organica


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",,,3
1,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc.",,,8
2,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc.",,,1
3,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc.",,,1
4,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc.",,,1
5,Jun 28 2022 2:58PM,240662,18,"ClearSpec, LLC",,,1
6,Jun 28 2022 2:53PM,240661,20,Alumier Labs Inc.,,,12
7,Jun 28 2022 2:33PM,240660,10,ISDIN Corporation,,3,10
8,Jun 28 2022 2:28PM,240658,18,"ClearSpec, LLC",,,1
9,Jun 28 2022 2:25PM,240657,12,Doctors Scientific Organica,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3,,
1,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc.",8,,
2,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc.",1,,
3,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc.",1,,
4,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc.",1,,
5,Jun 28 2022 2:58PM,240662,18,"ClearSpec, LLC",1,,
6,Jun 28 2022 2:53PM,240661,20,Alumier Labs Inc.,12,,
7,Jun 28 2022 2:33PM,240660,10,ISDIN Corporation,10,3,
8,Jun 28 2022 2:28PM,240658,18,"ClearSpec, LLC",1,,
9,Jun 28 2022 2:25PM,240657,12,Doctors Scientific Organica,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3,,
1,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc.",8,,
2,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc.",1,,
3,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc.",1,,
4,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3.0,NaN,NaN
1,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc.",8.0,NaN,NaN
2,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc.",1.0,NaN,NaN
4,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3.0,0.0,0.0
1,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc.",8.0,0.0,0.0
2,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc.",1.0,0.0,0.0
4,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2165763,31.0,22.0,0.0
12,240657,0.0,1.0,0.0
15,1203063,23.0,37.0,10.0
18,721934,3.0,0.0,0.0
19,962442,18.0,12.0,0.0
20,1443814,46.0,7.0,0.0
21,1203047,5.0,0.0,0.0
22,1443844,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2165763,31.0,22.0,0.0
1,12,240657,0.0,1.0,0.0
2,15,1203063,23.0,37.0,10.0
3,18,721934,3.0,0.0,0.0
4,19,962442,18.0,12.0,0.0
5,20,1443814,46.0,7.0,0.0
6,21,1203047,5.0,0.0,0.0
7,22,1443844,6.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,22.0,0.0
1,12,0.0,1.0,0.0
2,15,23.0,37.0,10.0
3,18,3.0,0.0,0.0
4,19,18.0,12.0,0.0
5,20,46.0,7.0,0.0
6,21,5.0,0.0,0.0
7,22,6.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,12,Released,0.0
2,15,Released,23.0
3,18,Released,3.0
4,19,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
Executing,22.0,1.0,37.0,0.0,12.0,7.0,0.0,0.0
Released,31.0,0.0,23.0,3.0,18.0,46.0,5.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,22.0,1.0,37.0,0.0,12.0,7.0,0.0,0.0
2,Released,31.0,0.0,23.0,3.0,18.0,46.0,5.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,22.0,1.0,37.0,0.0,12.0,7.0,0.0,0.0
2,Released,31.0,0.0,23.0,3.0,18.0,46.0,5.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()